In [2]:
import pandas as pd
import requests

url = "https://raw.githubusercontent.com/alzatem204/Tennis-Predictions/main/General_Data_ATP_UK.csv"
response = requests.get(url)

with open("General_Data_ATP_UK.csv", "wb") as f:
    f.write(response.content)

df = pd.read_csv("General_Data_ATP_UK.csv")
display(df.head())

/tmp/ipython-input-2-602830675.py:10: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("General_Data_ATP_UK.csv")


,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,Total Games,Partido,Sets,Score,Game Difference,Games Winner,Games Loser,Total Sets,Sets Difference,Year
0,34.0,Eastbourne,Eastbourne International,6/28/2025,ATP250,Outdoor,Grass,The Final,3,Fritz T.,...,19,Fritz T. vs. Brooksby J.,2 - 0,7-5 6-1,7,13,6,2,2,2025
1,35.0,Mallorca,Mallorca Championships,6/28/2025,ATP250,Outdoor,Grass,The Final,3,Griekspoor T.,...,25,Griekspoor T. vs. Moutet C.,2 - 0,7-5 7-6,3,14,11,2,2,2025
2,34.0,Eastbourne,Eastbourne International,6/27/2025,ATP250,Outdoor,Grass,Semifinals,3,Brooksby J.,...,33,Brooksby J. vs. Humbert U.,2 - 1,6-7 6-4 6-4,3,18,15,3,1,2025
3,34.0,Eastbourne,Eastbourne International,6/27/2025,ATP250,Outdoor,Grass,Semifinals,3,Fritz T.,...,25,Fritz T. vs. Davidovich Fokina A.,2 - 1,6-3 3-6 6-1,5,15,10,3,1,2025
4,35.0,Mallorca,Mallorca Championships,6/27/2025,ATP250,Outdoor,Grass,Semifinals,3,Griekspoor T.,...,20,Griekspoor T. vs. Auger-Aliassime F.,2 - 0,6-4 6-4,4,12,8,2,2,2025


In [3]:
post_match_columns = [
    'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5',
    'Wsets', 'Lsets', 'Score', 'Sets', 'Total Sets',
    'Game Difference', 'Games Winner', 'Games Loser', 'Total Games', 'Sets Difference',
    'Comment', 'Partido'
]

# Columnas de odds que no se van a usar (dejamos solo AvgW y AvgL)
odds_to_remove = [
    'B365W', 'B365L', 'PSW', 'PSL', 'MaxW', 'MaxL',
    'Best Ranking?', 'Best Odds?'
]

# Columnas adicionales a eliminar (columnas de identificación)
additional_to_remove = ['ATP', 'Date', 'Year']

# Unir todas las columnas a eliminar
columns_to_drop = post_match_columns + odds_to_remove + additional_to_remove

# Eliminar columnas
df_clean = df.drop(columns=columns_to_drop, errors='ignore')

# Eliminar filas con valores nulos
df_clean = df_clean.dropna()

# Mostrar columnas finales y conteo de filas
print("Columnas finales del dataset limpio:")
print(df_clean.columns.tolist())
print(f"Total de filas después de limpieza: {df_clean.shape[0]}")

df_clean.head()


Columnas finales del dataset limpio:
['Location', 'Tournament', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts', 'AvgW', 'AvgL']
Total de filas después de limpieza: 26316


,Location,Tournament,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,AvgW,AvgL
0,Eastbourne,Eastbourne International,ATP250,Outdoor,Grass,The Final,3,Fritz T.,Brooksby J.,5.0,149.0,4635.0,405.0,1.28,3.64
1,Mallorca,Mallorca Championships,ATP250,Outdoor,Grass,The Final,3,Griekspoor T.,Moutet C.,34.0,83.0,1535.0,739.0,1.56,2.39
2,Eastbourne,Eastbourne International,ATP250,Outdoor,Grass,Semifinals,3,Brooksby J.,Humbert U.,149.0,20.0,405.0,2195.0,2.76,1.44
3,Eastbourne,Eastbourne International,ATP250,Outdoor,Grass,Semifinals,3,Fritz T.,Davidovich Fokina A.,5.0,28.0,4635.0,1770.0,1.34,3.27
4,Mallorca,Mallorca Championships,ATP250,Outdoor,Grass,Semifinals,3,Griekspoor T.,Auger-Aliassime F.,34.0,27.0,1535.0,1775.0,2.18,1.67


In [4]:
df_clean['target'] = 1

# Crear una copia invertida (simetría)
df_flipped = df_clean.copy()

# Especificar columnas a intercambiar para simular que el perdedor está en la posición del ganador
columns_to_swap = {
    'Winner': 'Loser',
    'Loser': 'Winner',
    'WRank': 'LRank',
    'LRank': 'WRank',
    'WPts': 'LPts',
    'LPts': 'WPts',
    'AvgW': 'AvgL',
    'AvgL': 'AvgW'
}

# Renombrar las columnas invertidas
df_flipped.rename(columns=columns_to_swap, inplace=True)

# Cambiar la columna target a 0 (ya que ahora está en la posición opuesta)
df_flipped['target'] = 0

# Unir el dataset original con el invertido
df_balanced = pd.concat([df_clean, df_flipped], ignore_index=True)

# Mezclar (shuffle) si se desea
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Verificar forma final
print("Dataset balanceado creado con éxito.")
print(f"Total de filas: {df_balanced.shape[0]}")
print(f"Columnas: {df_balanced.columns.tolist()}")

Dataset balanceado creado con éxito.
Total de filas: 52632
Columnas: ['Location', 'Tournament', 'Series', 'Court', 'Surface', 'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts', 'AvgW', 'AvgL', 'target']


In [5]:
df_balanced.head()

,Location,Tournament,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,AvgW,AvgL,target
0,Paris,BNP Paribas Masters,Masters 1000,Indoor,Hard,2nd Round,3,Zverev A.,Tiafoe F.,5.0,44.0,5115.0,1055.0,1.27,3.86,1
1,Cincinnati,Western & Southern Financial Group Masters,Masters 1000,Outdoor,Hard,2nd Round,3.0,Khachanov K.,Fabbiano T.,32.0,85.0,1320.0,626.0,1.22,4.29,1
2,Auckland,ASB Classic,ATP250,Outdoor,Hard,1st Round,3.0,Vesely J.,Albot R.,64.0,88.0,815.0,616.0,1.81,1.98,1
3,Los Cabos,Los Cabos Open,ATP250,Outdoor,Hard,1st Round,3,Purcell M.,Arnaldi M.,51.0,41.0,961.0,1061.0,2.40,1.56,1
4,Gstaad,Suisse Open Gstaad,ATP250,Outdoor,Clay,Quarterfinals,3.0,Gulbis E.,Fognini F.,313.0,31.0,150.0,1270.0,1.94,1.86,0


In [7]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['Location', 'Tournament', 'Series', 'Court', 'Surface', 'Round',
                    'Best of', 'Winner', 'Loser']

# Rellenar valores nulos con 'Unknown'
for col in categorical_cols:
    df_balanced[col] = df_balanced[col].fillna('Unknown')

# Aplicar LabelEncoder a cada columna categórica
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_balanced[col] = le.fit_transform(df_balanced[col].astype(str))
    label_encoders[col] = le  # guardamos el encoder por si se necesita decodificar luego

In [8]:
df_balanced.head()

,Location,Tournament,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,AvgW,AvgL,target
0,68,20,3,0,2,1,1,765,679,5.0,44.0,5115.0,1055.0,1.27,3.86,1
1,23,131,3,1,2,1,2,337,202,32.0,85.0,1320.0,626.0,1.22,4.29,1
2,7,4,0,1,2,0,2,720,3,64.0,88.0,815.0,616.0,1.81,1.98,1
3,46,70,0,1,2,0,1,559,22,51.0,41.0,961.0,1061.0,2.40,1.56,1
4,35,118,0,1,0,4,2,269,216,313.0,31.0,150.0,1270.0,1.94,1.86,0


In [9]:
df_balanced.to_csv("ATP_UK_Encoded.csv", index=False)